In [1]:
!pip install -q --upgrade keras

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

import keras
from keras.models import load_model
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras import backend as K
from sklearn.preprocessing import StandardScaler
from keras.optimizers import Adam
from keras.applications import ResNet50, ResNet152
from keras.applications.resnet50 import preprocess_input
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
def append_ext(fn):
    return fn+".jpeg"

In [4]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/CGIAR/Train.csv')
ss = pd.read_csv('/content/drive/My Drive/Colab Notebooks/CGIAR/SampleSubmission.csv')
img_path = '/content/drive/My Drive/Colab Notebooks/CGIAR/Images'

In [5]:
train = df.loc[(df.label_quality==2)]
train_1 = df.loc[(df.label_quality==1) & (df.growth_stage==1)].sample(n=50, random_state=42)
train_6 = df.loc[(df.label_quality==1) & (df.growth_stage==6)].sample(n=50, random_state=42)

train = train.append([train_1, train_6], ignore_index=True).sample(frac=1)

train['UID'] = train['UID'].apply(append_ext)
test = ss.loc[:]
test['UID'] = test['UID'].apply(append_ext)

In [6]:
test.shape

(3558, 2)

In [13]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,validation_split=0.2)

train_generator = train_datagen.flow_from_dataframe(train, directory=img_path, batch_size=32, x_col='UID', y_col='growth_stage',
                                                    class_mode='raw', target_size=(512,169), shuffle=True, seed=42,
                                                    subset='training')
valid_generator = train_datagen.flow_from_dataframe(train, directory=img_path, batch_size=32, x_col='UID', y_col='growth_stage',
                                                    class_mode='raw', target_size=(512,169), shuffle=True, seed=42,
                                                    subset='validation')

test_datagen = ImageDataGenerator(rescale=1./255.)
test_generator = test_datagen.flow_from_dataframe(dataframe=test, directory=img_path, batch_size=32, x_col='UID', y_col=None,
                                                  class_mode=None, target_size=(512,169), shuffle=False, seed=42)

Found 2365 validated image filenames.
Found 591 validated image filenames.
Found 3558 validated image filenames.


In [5]:

# train_images = np.zeros((len(train.UID),169,512,3))
# i = 0 
# for uid in (train['UID']):
#   img = img_path + '/' + uid + '.jpeg'
#   a = Image.open(img).resize((512,169))
#   train_images[i] = np.asarray(a)
#   i += 1




In [6]:
# train_labels = np.array(train.growth_stage)

In [7]:

# binary encoding
binary_encoder = OneHotEncoder(sparse=False)
train_labels_encoded = binary_encoder.fit_transform(train_labels.reshape(-1,1))


In [14]:
base_model = ResNet50(weights='imagenet',  # Load weights pre-trained on ImageNet.
                      input_shape=(169, 512, 3), include_top=False)  # Do not include the ImageNet classifier at the top.
base_model.trainable = False

In [17]:
inputs = keras.Input(shape=(169, 512, 3))
# We make sure that the base_model is running in inference mode here,
# by passing `training=False`. This is important for fine-tuning, as you will
# learn in a few paragraphs.
x = base_model(inputs, training=False)
# Convert features of shape `base_model.output_shape[1:]` to vectors
x = keras.layers.GlobalAveragePooling2D()(x)
# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

In [25]:
EPOCHS = 10
INIT_LR = 1e-3
BS = 32

# initialize the optimizer (SGD is sufficient)
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)

model.compile(optimizer=opt,
              loss=keras.losses.MeanSquaredError(),
              metrics=keras.metrics.MeanSquaredError())
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size

model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=EPOCHS
)
# model.fit(train_images, train_labels_encoded, epochs=EPOCHS, batch_size=BS, verbose=1, validation_split=0.2,
#           use_multiprocessing=True)

Epoch 1/10
73/73 [==============================] - 74s 1s/step - loss: 2.1173 - mean_squared_error: 2.1173 - val_loss: 1.9311 - val_mean_squared_error: 1.9311
Epoch 2/10
73/73 [==============================] - 72s 993ms/step - loss: 2.1070 - mean_squared_error: 2.1070 - val_loss: 1.8406 - val_mean_squared_error: 1.8406
Epoch 3/10
73/73 [==============================] - 72s 989ms/step - loss: 2.0666 - mean_squared_error: 2.0666 - val_loss: 1.8702 - val_mean_squared_error: 1.8702
Epoch 4/10
73/73 [==============================] - 72s 991ms/step - loss: 1.9871 - mean_squared_error: 1.9871 - val_loss: 1.8409 - val_mean_squared_error: 1.8409
Epoch 5/10
73/73 [==============================] - 72s 986ms/step - loss: 2.0246 - mean_squared_error: 2.0246 - val_loss: 1.8663 - val_mean_squared_error: 1.8663
Epoch 6/10
73/73 [==============================] - 71s 979ms/step - loss: 2.0089 - mean_squared_error: 2.0089 - val_loss: 1.7825 - val_mean_squared_error: 1.7825
Epoch 7/10
73/73 [=======

In [23]:
model.evaluate(x=valid_generator,steps=STEP_SIZE_TEST,)

 19/111 [====>.........................] - 14s 727ms/step - loss: 4.6292e-07 - accuracy: 0.0000e+00


[4.629193597338599e-07, 0.0]